In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm 


import pandas as pd

path='C:\\Users\\HP\\PycharmProjects\\mds\\hb_mc\\'
data = pd.read_csv(path+'radon.csv')

county_names = data.county.unique()
county_idx = data['county_code'].values
n_counties = len(data.county.unique())
data[['county', 'log_radon', 'floor']].head()

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


,county,log_radon,floor
0,AITKIN,0.832909,1.0
1,AITKIN,0.832909,0.0
2,AITKIN,1.098612,0.0
3,AITKIN,0.095310,0.0
4,ANOKA,1.163151,0.0


In [10]:
indiv_traces = {}
for county_name in county_names:
    # Select subset of data belonging to county
    c_data = data.ix[data.county == county_name]
    c_log_radon = c_data.log_radon
    c_floor_measure = c_data.floor.values
    
    with pm.Model() as individual_model:
        # Intercept prior (variance == sd**2)
        a = pm.Normal('alpha', mu=0, sd=100**2)
        # Slope prior
        b = pm.Normal('beta', mu=0, sd=100**2)
    
        # Model error prior
        eps = pm.Uniform('eps', lower=0, upper=100)
    
        # Linear model
        radon_est = a + b * c_floor_measure
    
        # Data likelihood
        radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=c_log_radon)

        # Inference button (TM)!
        step = pm.NUTS()
        trace = pm.sample(2000, step=step, progressbar=False)
    
    # keep trace for later analysis
    indiv_traces[county_name] = trace

Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.


Applied interval-transform to eps and added transformed eps_interval_ to model.



KeyboardInterrupt


In [12]:
indiv_traces

{'AITKIN': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'ANOKA': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BECKER': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BELTRAMI': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BENTON': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BIG STONE': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BLUE EARTH': <MultiTrace: 1 chains, 2000 iterations, 4 variables>,
 'BROWN': <MultiTrace: 1 chains, 136 iterations, 4 variables>}

In [14]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_alpha', mu=0., sd=100**2)
    sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
    mu_b = pm.Normal('mu_beta', mu=0., sd=100**2)
    sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=n_counties)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=n_counties)
    
    # Model error
    eps = pm.Uniform('eps', lower=0, upper=100)
    
    # Model prediction of radon level
    # a[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county
    # to its coefficients.
    radon_est = a[county_idx] + b[county_idx] * data.floor.values
    
    # Data likelihood
    radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=data.log_radon)

Applied interval-transform to sigma_alpha and added transformed sigma_alpha_interval_ to model.
Applied interval-transform to sigma_beta and added transformed sigma_beta_interval_ to model.
Applied interval-transform to eps and added transformed eps_interval_ to model.


In [ ]:
# Inference button (TM)!
with hierarchical_model:
    # Use ADVI for initialization
    mu, sds, elbo = pm.variational.advi(n=100000)
    step = pm.NUTS(scaling=hierarchical_model.dict_to_array(sds)**2, 
                   is_cov=True)
    hierarchical_trace = pm.sample(5000, step, start=mu)

In [ ]:
# Plotting the hierarchical model trace -its found values- from 500 iterations onwards (right side plot) 
# and its accumulated marginal values (left side plot) 
pm.traceplot(hierarchical_trace[500:]);

In [1]:
2+2

4